In [1]:
import json
import pickle
import random
import nltk
import numpy
from nltk.stem import LancasterStemmer
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.models import model_from_yaml

In [2]:
nltk.download('punkt')

stemmer = LancasterStemmer()

with open("intents.json") as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
try:
    with open("chatbot.pickle", "rb") as file:
        words, labels, training, output = pickle.load(file)

except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    output_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = output_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("chatbot.pickle", "wb") as file:
        pickle.dump((words, labels, training, output), file)

In [4]:
try:
    yaml_file = open('chatbotmodel.yaml', 'r')
    loaded_model_yaml = yaml_file.read()
    yaml_file.close()
    myChatModel = model_from_yaml(loaded_model_yaml)
    myChatModel.load_weights("chatbotmodel.h5")
    print("Loaded model from disk")


except:
    # make our neural network
    myChatModel = Sequential()
    myChatModel.add(Dense(8, input_shape=[len(words)], activation='relu'))
    myChatModel.add(Dense(len(labels), activation='softmax'))

    # optimize the model
    myChatModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # train the model
    myChatModel.fit(training, output, epochs=1000, batch_size=8)

    # serialize model to yaml and save it to disk
    model_yaml = myChatModel.to_yaml()
    with open("chatbotmodel.yaml", "w") as y_file:
        y_file.write(model_yaml)

    # serialize weights to HDF5
    myChatModel.save_weights("chatbotmodel.h5")
    print("Saved model to disk")

Epoch 1/1000
4/4 [==============================] - 0s 0s/step - loss: 1.8209 - accuracy: 0.1154
Epoch 2/1000
4/4 [==============================] - 0s 0s/step - loss: 1.8103 - accuracy: 0.1154 
Epoch 3/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.8003 - accuracy: 0.0769
Epoch 4/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.7924 - accuracy: 0.0769
Epoch 5/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.7840 - accuracy: 0.0769
Epoch 6/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.7755 - accuracy: 0.1154
Epoch 7/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.7677 - accuracy: 0.1154
Epoch 8/1000
4/4 [==============================] - 0s 0s/step - loss: 1.7591 - accuracy: 0.1154 
Epoch 9/1000
4/4 [==============================] - 0s 0s/step - loss: 1.7520 - accuracy: 0.1154
Epoch 10/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.7438 - accuracy: 0.1154
Epoch 11/1000
4/4 [==

4/4 [==============================] - 0s 0s/step - loss: 1.1838 - accuracy: 0.6154
Epoch 85/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.1757 - accuracy: 0.6154
Epoch 86/1000
4/4 [==============================] - 0s 0s/step - loss: 1.1675 - accuracy: 0.6154
Epoch 87/1000
4/4 [==============================] - 0s 0s/step - loss: 1.1596 - accuracy: 0.6154
Epoch 88/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.1520 - accuracy: 0.6154
Epoch 89/1000
4/4 [==============================] - 0s 0s/step - loss: 1.1433 - accuracy: 0.6154
Epoch 90/1000
4/4 [==============================] - 0s 2ms/step - loss: 1.1352 - accuracy: 0.6154
Epoch 91/1000
4/4 [==============================] - 0s 0s/step - loss: 1.1269 - accuracy: 0.6154
Epoch 92/1000
4/4 [==============================] - 0s 0s/step - loss: 1.1190 - accuracy: 0.6154
Epoch 93/1000
4/4 [==============================] - 0s 4ms/step - loss: 1.1101 - accuracy: 0.6154
Epoch 94/1000
4/4 [===========

4/4 [==============================] - 0s 4ms/step - loss: 0.6548 - accuracy: 0.9231
Epoch 167/1000
4/4 [==============================] - 0s 0s/step - loss: 0.6501 - accuracy: 0.9231
Epoch 168/1000
4/4 [==============================] - 0s 0s/step - loss: 0.6450 - accuracy: 0.9231
Epoch 169/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.6399 - accuracy: 0.9231
Epoch 170/1000
4/4 [==============================] - 0s 0s/step - loss: 0.6350 - accuracy: 0.9231
Epoch 171/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.6298 - accuracy: 0.9231
Epoch 172/1000
4/4 [==============================] - 0s 0s/step - loss: 0.6244 - accuracy: 0.9615
Epoch 173/1000
4/4 [==============================] - 0s 0s/step - loss: 0.6193 - accuracy: 0.9615
Epoch 174/1000
4/4 [==============================] - ETA: 0s - loss: 0.5361 - accuracy: 1.00 - 0s 4ms/step - loss: 0.6144 - accuracy: 0.9615
Epoch 175/1000
4/4 [==============================] - 0s 0s/step - loss: 0.609

4/4 [==============================] - 0s 0s/step - loss: 0.3272 - accuracy: 1.0000
Epoch 247/1000
4/4 [==============================] - 0s 0s/step - loss: 0.3247 - accuracy: 1.0000
Epoch 248/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.3222 - accuracy: 1.0000
Epoch 249/1000
4/4 [==============================] - 0s 0s/step - loss: 0.3192 - accuracy: 1.0000
Epoch 250/1000
4/4 [==============================] - 0s 0s/step - loss: 0.3164 - accuracy: 1.0000
Epoch 251/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.3136 - accuracy: 1.0000
Epoch 252/1000
4/4 [==============================] - 0s 0s/step - loss: 0.3109 - accuracy: 1.0000
Epoch 253/1000
4/4 [==============================] - 0s 0s/step - loss: 0.3079 - accuracy: 1.0000
Epoch 254/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.3055 - accuracy: 1.0000
Epoch 255/1000
4/4 [==============================] - 0s 0s/step - loss: 0.3028 - accuracy: 1.0000
Epoch 256/1000
4/4 [==

4/4 [==============================] - 0s 0s/step - loss: 0.1643 - accuracy: 1.0000
Epoch 329/1000
4/4 [==============================] - 0s 0s/step - loss: 0.1630 - accuracy: 1.0000
Epoch 330/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1615 - accuracy: 1.0000
Epoch 331/1000
4/4 [==============================] - 0s 0s/step - loss: 0.1602 - accuracy: 1.0000
Epoch 332/1000
4/4 [==============================] - 0s 0s/step - loss: 0.1591 - accuracy: 1.0000
Epoch 333/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1576 - accuracy: 1.0000
Epoch 334/1000
4/4 [==============================] - 0s 0s/step - loss: 0.1563 - accuracy: 1.0000
Epoch 335/1000
4/4 [==============================] - 0s 0s/step - loss: 0.1548 - accuracy: 1.0000
Epoch 336/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1533 - accuracy: 1.0000
Epoch 337/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.1520 - accuracy: 1.0000
Epoch 338/1000
4/4 [=

...

4/4 [==============================] - 0s 0s/step - loss: 0.0187 - accuracy: 1.0000
Epoch 657/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0186 - accuracy: 1.0000
Epoch 658/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0185 - accuracy: 1.0000
Epoch 659/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0184 - accuracy: 1.0000
Epoch 660/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0184 - accuracy: 1.0000
Epoch 661/1000
4/4 [==============================] - 0s 510us/step - loss: 0.0183 - accuracy: 1.0000
Epoch 662/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0182 - accuracy: 1.0000
Epoch 663/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0181 - accuracy: 1.0000
Epoch 664/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0180 - accuracy: 1.0000
Epoch 665/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0179 - accuracy: 1.0000
Epoch 666/1000
4/4 

4/4 [==============================] - 0s 0s/step - loss: 0.0128 - accuracy: 1.0000
Epoch 739/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 740/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 741/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0126 - accuracy: 1.0000
Epoch 742/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0126 - accuracy: 1.0000
Epoch 743/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0125 - accuracy: 1.0000
Epoch 744/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0125 - accuracy: 1.0000
Epoch 745/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 746/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0124 - accuracy: 1.0000
Epoch 747/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0123 - accuracy: 1.0000
Epoch 748/1000
4/4 [=

4/4 [==============================] - 0s 4ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 820/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 821/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0092 - accuracy: 1.0000
Epoch 822/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0091 - accuracy: 1.0000
Epoch 823/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0091 - accuracy: 1.0000
Epoch 824/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0091 - accuracy: 1.0000
Epoch 825/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0090 - accuracy: 1.0000
Epoch 826/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0090 - accuracy: 1.0000
Epoch 827/1000
4/4 [==============================] - 0s 2ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 828/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0089 - accuracy: 1.0000
Epoch 829/1000
4/4 [=

4/4 [==============================] - 0s 0s/step - loss: 0.0068 - accuracy: 1.0000
Epoch 901/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0068 - accuracy: 1.0000
Epoch 902/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0067 - accuracy: 1.0000
Epoch 903/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 904/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 905/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 906/1000
4/4 [==============================] - 0s 6ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 907/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 908/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 909/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 910/1000
4/4

4/4 [==============================] - 0s 0s/step - loss: 0.0051 - accuracy: 1.0000
Epoch 983/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0051 - accuracy: 1.0000
Epoch 984/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0051 - accuracy: 1.0000
Epoch 985/1000
4/4 [==============================] - 0s 1ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 986/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 987/1000
4/4 [==============================] - 0s 4ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 988/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0051 - accuracy: 1.0000
Epoch 989/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0050 - accuracy: 1.0000
Epoch 990/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0050 - accuracy: 1.0000
Epoch 991/1000
4/4 [==============================] - 0s 0s/step - loss: 0.0050 - accuracy: 1.0000
Epoch 992/1000
4/4 [==

In [5]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

def chatWithBot(inputText):

    currentText = bag_of_words(inputText, words)
    currentTextArray = [currentText]
    numpyCurrentText = numpy.array(currentTextArray)

    if numpy.all((numpyCurrentText == 0)):
        return "I didn't get that, try again"

    result = myChatModel.predict(numpyCurrentText[0:1])
    result_index = numpy.argmax(result)
    tag = labels[result_index]

    if result[0][result_index] > 0.7:
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        return random.choice(responses)

    else:
        return "I didn't get that, try again"

def chat():
    print("Start talking with the chatbox (type quit to stop)")

    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            print("Its really sad to see you leave")
            break

        print(chatWithBot(inp))

In [ ]:
chat()

Start talking with the chatbox (type quit to stop)
